In [25]:
# Set up imports
import pandas as pd
import numpy as np
import re
import sys

In [26]:
# Load dataset
init_df = pd.read_csv(r"Standardized Data\Final.csv")
df = init_df.copy()

In [27]:
# Look through and check data
df.drop(columns=["Unnamed: 0"], inplace=True)

In [28]:
# Drop rows with empty text
df.dropna(subset=["Text"], inplace=True)
df.reset_index(drop = True)
df.columns

Index(['Entity Type', 'Entity Name', 'City, State', 'Enforcement Action',
       'Date', 'Text', 'State Enforcing', 'Text Length'],
      dtype='object')

In [45]:
# Add a new column to check if Text has been processed
df["Processed?"] = False

In [46]:
df.head()

,Entity Type,Entity Name,"City, State",Enforcement Action,Date,Text,State Enforcing,Text Length,Processed?
4,RM,"T. P. O. T. T., INC.","COLUMBUS, OH",NOTICE OF INTENT TO REVOKE,04/18/2001,STATE OF OHIO DEPARTMENT OF COMMERCE DIVISION ...,OH,8761,False
5,RM,"T. P. O. T. T., INC.","COLUMBUS, OH",REPORT AND RECOMMENDATION,01/22/2002,STALE OF Ohio Department of Commerce Bob Taft ...,OH,35918,False
6,RM,"T. P. O. T. T., INC.","COLUMBUS, OH",DIVISION ORDER,02/19/2002,STATE OF OHIO DEPARTMENT OF COMMERCE DIVISION ...,OH,4314,False
7,RM,ABC FINANCIAL SERVICES CORPORATION,"CORAOPOLIS, PA",DIVISION ORDER,02/26/2002,Ohio Department of Commerce Division of Financ...,OH,1981,False
8,RM,"MORTGAGE SPECIALISTS, INC.","NICHOLASVILLE, KY",DIVISION ORDER,02/26/2002,STATE OF OHIO DEPARTMENT OF COMMERCE DIVISION ...,OH,3259,False


## Beginning of Filtering Process

In [49]:
# Function to extract text between two strings (CAPS SENSITIVE)
def extract_text(row, sub1, sub2):
    text = row["Text"]
    if sub1 in text and sub2 in text:
        index_1 = text.index(sub1)
        index_2 = text.index(sub2)
        row["Text"] = text[index_1 + len(sub1): index_2]
        row["Processed?"] = True
    return row
    

In [61]:
# Check how many rows remain unprocessed
## USE THIS TO CHECK HOW MANY ROWS YOU HAVE LEFT
df.loc[(df["State Enforcing"]=="OH")& (df["Processed?"]==False)].shape

(4100, 9)

In [50]:
# Function to Filter Text
def filter_text(df, start_term, end_term, state):
    df = df.apply(lambda row: extract_text(row, start_term, end_term) if (row["Processed?"]== False) & (row["State Enforcing"]==state)  else row, axis = 1)
    print("processed")
    return df


In [60]:
# Cleaning of Text

# THIS FUNCTION FILTERS START AND END, JUST KIND OF COPY THE SYNTAX AND KEEP GOING DOWN.

# BASIS FOR PROPOSED ACTION: 2207 hits
# NOTICE OF HEARING: 201 hits
df = filter_text(df, "BASIS FOR PROPOSED ACTION", "NOTICE OF HEARING", "OH")
# NOTICE OF OPPORTUNITY: 2207 hits
df = filter_text(df, "BASIS FOR PROPOSED ACTION", "NOTICE OF OPPORTUNITY", "OH")

# STIPULATIONS AND ADMISSIONS: 1031 hits
# ACKNOWLEDGEMENTS/LIABILITY RELEASE:
df.loc[(df["Text"].str.contains("STIPULATIONS AND ADMISSIONS")) & (df["State Enforcing"]=="OH") & (df["Text"].str.contains("AGREED CONDITIONS"))& (df["Processed?"]==False)].describe()
# df.loc[(df["Text"].str.contains("STIPULATIONS AND ADMISSIONS")) & (df["State Enforcing"]=="OH") & (df["Processed?"]==False)].describe()

processed
processed


,Text Length
count,1023.000000
mean,8627.347019
std,1605.645992
min,6188.000000
25%,7358.000000
50%,8052.000000
75%,9751.000000
max,15451.000000
